## joining all the data in silver layer and filtering the columns thats needed for aggregation and storing it in gold layer layer for aggregations and visualisations

####  using run commands to access notebook to run codes to configuring the adlsgen2 and connect

In [0]:
%run "../includes/adlsconfig"

In [0]:
%run "../includes/containersconfig"

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

#Races File

In [0]:
races_df = spark.read.format('parquet').option("header", True).option('inferSchema', True).load(f"{silver_folder_path}/races")

In [0]:
races_df.display()

# circuits file

In [0]:
circuits_df = spark.read.format('parquet').option("header", True).option('inferSchema', True).load(f"{silver_folder_path}/circuits")

In [0]:
circuits_df.display()

#races file

In [0]:
results_df = spark.read.format('parquet').option("header", True).option('inferSchema', True).load(f"{silver_folder_path}/results")

In [0]:
results_df.display()

#drivers file

In [0]:
drivers_df = spark.read.format('parquet').option("header", True).option('inferSchema', True).load(f"{silver_folder_path}/drivers")

In [0]:
drivers_df.display()

#constructors file

In [0]:
constructors_df = spark.read.format('parquet').option("header", True).option('inferSchema', True).load(f"{silver_folder_path}/constructors")

In [0]:
constructors_df.display()

In [0]:

## all the data files are joined into a single data from to be written into gold layer
joins_final  =  (results_df.join(races_df, results_df.race_id == races_df.race_id, "inner")\
    .join(drivers_df, results_df.driver_id == drivers_df.driver_id, "inner")\
        .join(constructors_df, results_df.constructor_id == constructors_df.constructor_id, "inner")\
            .join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, "inner")\
                .select(races_df.race_year,
                        races_df.name.alias('race_name'),
                        races_df.race_timestamp.alias('race_date'),
                        circuits_df.location,
                        drivers_df.name.alias('drivers_name'),
                        drivers_df.number.alias('drivers_number'),
                        drivers_df.nationality,
                        constructors_df.name.alias('team_name'),
                        results_df.position ,
                        results_df.grid,
                        results_df[" fastest_lap"],
                        results_df.time.alias('race_time'),
                        results_df.points
                        )\
                            .withColumn("created_date", current_timestamp())
                                
                )

In [0]:
joins_final.display()

In [0]:
joins_final.write.mode('overwrite').format('parquet').save(f"{gold_folder_path}/race_results")

In [0]:
df = spark.read.format('parquet').load(f"{gold_folder_path}/race_results").display()